In [1]:
!git clone https://github.com/OpenGVLab/InternVL.git

Cloning into 'InternVL'...
remote: Enumerating objects: 3445, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 3445 (delta 336), reused 269 (delta 269), pack-reused 3045 (from 2)
Receiving objects: 100% (3445/3445), 39.45 MiB | 12.72 MiB/s, done.
Resolving deltas: 100% (2056/2056), done.


In [3]:
# !huggingface-cli download --resume-download --local-dir-use-symlinks False OpenGVLab/InternVL2_5-1B-MPO --local-dir InternVL2_5-1B-MPO

# Dataset Creation

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00


In [35]:
# # hugging face dataset download
from datasets import load_dataset
dataset = load_dataset("your_dataset", token="input_token", split='train[:100]')

In [6]:

dataset

Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 100
})

In [8]:
# dataset["image"]

In [9]:
import os
from PIL import Image
import requests
from io import BytesIO

# Step 1: Create dataset folder and image folder
dataset_folder = "dataset_folder"
image_folder = os.path.join(dataset_folder, "images")

os.makedirs(image_folder, exist_ok=True)

# Step 2: Save images to the 'images' folder
for idx, sample in enumerate(dataset):
    # Assuming dataset has an 'image' column with PIL Image or URL
    if isinstance(sample['image'], Image.Image):
        image = sample['image']
    else:
        # If it's a URL, download the image
        response = requests.get(sample['image']['url'])
        image = Image.open(BytesIO(response.content))

    # Save the image
    image_path = os.path.join(image_folder, f"image_{idx}.jpg")
    image.save(image_path)
    # print(f"Saved: {image_path}")

print("All images have been saved successfully!")


All images have been saved successfully!


In [10]:
final_dataset_json = []
relative_image_path = "/content/dataset_folder/images"
for idx, sample in enumerate(dataset):
    temp_json_dataset = {}

    x, y = dataset["image"][idx].size
    # Assuming dataset has an 'image' column with PIL Image or URL
    # print(sample["question"], sample["answer"])


    temp_json_dataset["id"] = idx
    temp_json_dataset["image"] = f"{relative_image_path}/image_{idx}.jpg"
    temp_json_dataset["width"] = x
    temp_json_dataset["height"] = y
    temp_json_dataset["conversations"] = [
      {
        "from": "human",
        "value": f"<image>\n{sample['question']}"
      },
      {
        "from": "gpt",
        "value": f"{sample['answer']}"
      }
    ]
    final_dataset_json.append(temp_json_dataset)

    # if idx == 2:
    #   break

    # break

In this format json file required
```
{
  "id": 0,
  "image": "images/00000000.jpg",
  "width": 897,
  "height": 1152,
  "conversations": [
    {
      "from": "human",
      "value": "<image>\nCan you extract any readable text from the image?"
    },
    {
      "from": "gpt",
      "value": "Dares Wins Vol. 5 Tommy's Heroes Vol. 6: For Tomorrow Vol. 7: Closing Time miniseries. Clark Kent is being interviewed about Superman's connection to notorious killer Tommy Monaghan. Taking the conversation..."
    }
  ]
}
```

In [11]:
# final_dataset_json

In [12]:
import json
with open('data.json', 'w') as jsonl_file:
    for item in final_dataset_json:
        # Convert the dictionary to a JSON string
        json_line = json.dumps(item)

        # Write the JSON string followed by a newline
        jsonl_file.write(json_line + '\n')

In [13]:
# creating meta_data_file for data information
import json

meta_data = {
    "data": {
        "root": "/content/dataset_folder/images",
        "annotation": "/content/data.json",
        "data_augment": False,
        "repeat_time": 1,
        "length": len(dataset)
    }
}
with open("/content/InternVL/internvl_chat/shell/data/train_metadata.json", "w") as f:
    json.dump(meta_data, f)

In [14]:
import sys
import os

# Get the current script's directory or set a custom path
# current_path = os.path.abspath("/content/InternVL")

# # Add the directory to sys.path
# if current_path not in sys.path:
#     sys.path.append(current_path)

sys.path.append("/content/InternVL")

In [15]:
os.chdir('/content/InternVL/internvl_chat')

In [16]:
!pwd

/content/InternVL/internvl_chat


In [17]:
# !pip install flash-attn==2.0.2

In [18]:
!pip install -r /content/InternVL/requirements/internvl_chat.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━

In [19]:
!pip install deepspeed

In [20]:
# # !pip install flash_attn exllamav2 transforme  r -U
# !pip install psutil
# !pip install flash_attn --no-build-isolation

In [21]:
!nvidia-smi

Thu Feb 20 03:47:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
# !GPUS=1 PER_DEVICE_BATCH_SIZE=1 sh /content/InternVL/internvl_chat/shell/internvl2.5_mpo/preference_optimization/internvl2_5_1b_qwen2_5_0_5b_dynamic_res_mpo.sh

In [23]:
!pip3 install ipywidgets jupyterlab
!pip3 install -q -U bitsandbytes
!pip3 install -q -U git+https://github.com/huggingface/transformers.git
!pip3 install -q -U git+https://github.com/huggingface/peft.git
!pip3 install -q -U git+https://github.com/huggingface/accelerate.git
!pip3 install -q -U git+https://github.com/Dao-AILab/flash-attention.git
!pip3 install -q -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.24.0
    Uninstalling jupyter-server-1.24.0:
      Successfully uninstalled jupyter-server-1.24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.6 MB/s eta 0:00

In [24]:
!pip install flash_attn exllamav2 transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.5 MB/s eta 0:00:00


In [25]:
# !pip install --upgrade transformers

In [26]:
# !pip uninstall flash-attn
# !pip install flash-attn --no-build-isolation --no-cache-dir

In [27]:
import torch
from transformers import AutoTokenizer, AutoModel
path = "OpenGVLab/InternVL2_5-1B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_intern_vit.py:   0%|          | 0.00/18.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- modeling_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- modeling_internvl_chat.py
- conversation.py
- modeling_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [28]:
import transformers
print(transformers.__version__)

4.50.0.dev0


In [29]:
!sed -i 's#\./shell/data/internvl_1_2_finetune_custom.json#/content/InternVL/internvl_chat/shell/data/train_metadata.json#g' /content/InternVL/internvl_chat/shell/internvl2.5/2nd_finetune/internvl2_5_1b_dynamic_res_2nd_finetune_lora.sh


In [30]:
!GPUS=1 PER_DEVICE_BATCH_SIZE=1 sh /content/InternVL/internvl_chat/shell/internvl2.5/2nd_finetune/internvl2_5_1b_dynamic_res_2nd_finetune_lora.sh

+ GPUS=1
+ BATCH_SIZE=16
+ PER_DEVICE_BATCH_SIZE=1
+ GRADIENT_ACC=16
+ pwd
+ export PYTHONPATH=/env/python:/content/InternVL/internvl_chat
+ export MASTER_PORT=34229
+ export TF_CPP_MIN_LOG_LEVEL=3
+ export LAUNCHER=pytorch
+ OUTPUT_DIR=work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora
+ [ ! -d work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora ]
+ mkdir -p work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora
+ torchrun --nnodes=1 --node_rank=0 --master_addr=127.0.0.1 --nproc_per_node=1 --master_port=34229 internvl/train/internvl_chat_finetune.py --model_name_or_path OpenGVLab/InternVL2_5-1B --conv_style internvl2_5 --use_fast_tokenizer False --output_dir work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora --meta_path /content/InternVL/internvl_chat/shell/data/train_metadata.json --overwrite_output_dir True --force_image_size 448 --max_dynamic_patch 6 --down_sample_ratio 0.5 --drop_path_rate 0.0 -

In [31]:
# from transformers.models.llama.modeling_llama import LLAMA_ATTENTION_CLASSES

In [32]:
import transformers.models.llama.modeling_llama

In [33]:
import inspect
import transformers.models.llama.modeling_llama as llama

# List all functions
functions = [name for name, obj in inspect.getmembers(llama, inspect.isfunction)]
print("📘 Functions:", functions)

# List all classes
classes = [name for name, obj in inspect.getmembers(llama, inspect.isclass)]
print("🏛️ Classes:", classes)


📘 Functions: ['add_code_sample_docstrings', 'add_start_docstrings', 'add_start_docstrings_to_model_forward', 'apply_rotary_pos_emb', 'deprecate_kwarg', 'eager_attention_forward', 'repeat_kv', 'replace_return_docstrings', 'rotate_half']
🏛️ Classes: ['AttentionMaskConverter', 'BaseModelOutputWithPast', 'Cache', 'CausalLMOutputWithPast', 'DynamicCache', 'FlashAttentionKwargs', 'GenerationMixin', 'KwargsForCausalLM', 'LlamaAttention', 'LlamaConfig', 'LlamaDecoderLayer', 'LlamaForCausalLM', 'LlamaForQuestionAnswering', 'LlamaForSequenceClassification', 'LlamaForTokenClassification', 'LlamaMLP', 'LlamaModel', 'LlamaPreTrainedModel', 'LlamaRMSNorm', 'LlamaRotaryEmbedding', 'LossKwargs', 'PreTrainedModel', 'QuestionAnsweringModelOutput', 'SequenceClassifierOutputWithPast', 'StaticCache', 'TokenClassifierOutput']


In [34]:
!lmdeploy check_env

/bin/bash: line 1: lmdeploy: command not found
